<h2>Importing needed libraries</h2>

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

<h2>Dataset Organizing </h2>

In [21]:
# paths to training and validation folders
train_dir = './data/Bridge_Crack_Image/DBCC_Training_Data_Set/train'
validation_dir = './data/Bridge_Crack_Image/DBCC_Training_Data_Set/val'


In [24]:
import os

os.makedirs(os.path.join(train_dir, 'bridge_crack'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'no_crack'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'bridge_crack'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'no_crack'), exist_ok=True)

In [25]:
import shutil

def organize_images_by_name(source_dir, crack_dir, no_crack_dir):
    for filename in os.listdir(source_dir):
        if filename.lower().startswith("crack"):
            shutil.move(os.path.join(source_dir, filename), crack_dir)
        elif filename.lower().startswith("no"):
            shutil.move(os.path.join(source_dir, filename), no_crack_dir)

# Organize training images
organize_images_by_name(train_dir, os.path.join(train_dir, 'bridge_crack'), os.path.join(train_dir, 'no_crack'))

# Organize validation images
organize_images_by_name(val_dir, os.path.join(val_dir, 'bridge_crack'), os.path.join(val_dir, 'no_crack'))

<h2>Verifying the dataset</h2>

In [26]:
for root, dirs, files in os.walk('./data/Bridge_Crack_Image/DBCC_Training_Data_Set'):
    print(f'In {root}, found {len(files)} files and {len(dirs)} subdirectories')

In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set, found 2 files and 2 subdirectories
In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set\train, found 0 files and 2 subdirectories
In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set\train\bridge_crack, found 6000 files and 0 subdirectories
In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set\train\no_crack, found 44000 files and 0 subdirectories
In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set\val, found 0 files and 2 subdirectories
In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set\val\bridge_crack, found 1000 files and 0 subdirectories
In ./data/Bridge_Crack_Image/DBCC_Training_Data_Set\val\no_crack, found 4000 files and 0 subdirectories


<h2>Pre-Processing</h2>

In [27]:
# Define ImageDataGenerator with augmentations for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load training images
train_generator = train_datagen.flow_from_directory(
    './data/Bridge_Crack_Image/DBCC_Training_Data_Set/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Load validation images
validation_generator = validation_datagen.flow_from_directory(
    './data/Bridge_Crack_Image/DBCC_Training_Data_Set/val',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 50000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


<h2>Model creation (Neural-Network)</h2>

In [28]:
# creating a sequential neural network
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\Lenovo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<h2>Training Model</h2>

In [29]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/15


C:\Users\Lenovo\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.8721 - loss: 0.3490 - val_accuracy: 0.9775 - val_loss: 0.0698
Epoch 2/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.9642 - loss: 0.1499 - val_accuracy: 0.9625 - val_loss: 0.1264
Epoch 3/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.9771 - loss: 0.0953 - val_accuracy: 0.9887 - val_loss: 0.0492
Epoch 4/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 98s 977ms/step - accuracy: 0.9748 - loss: 0.1061 - val_accuracy: 0.9650 - val_loss: 0.1032
Epoch 5/15


C:\Users\Lenovo\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.9724 - loss: 0.1255 - val_accuracy: 0.9869 - val_loss: 0.0357
Epoch 6/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.9748 - loss: 0.1247 - val_accuracy: 0.9719 - val_loss: 0.1093
Epoch 7/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.9818 - loss: 0.0808 - val_accuracy: 0.9875 - val_loss: 0.0481
Epoch 8/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 93s 932ms/step - accuracy: 0.9851 - loss: 0.0664 - val_accuracy: 0.9800 - val_loss: 0.1108
Epoch 9/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.9585 - loss: 0.1579 - val_accuracy: 0.9806 - val_loss: 0.0684
Epoch 10/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.9773 - loss: 0.1056 - val_accuracy: 0.9869 - val_loss: 0.0502
Epoch 11/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 116s 1s/step - accuracy: 0.9779 - loss: 0.1067 - val_accuracy: 0.9844 - val_loss: 0.0626
Epoch 12/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.9814 - loss: 0.0839 - val_accuracy: 

<h2>Prediction</h2>

In [31]:
Y_pred = model.predict(validation_generator)
y_pred = np.round(Y_pred).astype(int)

157/157 ━━━━━━━━━━━━━━━━━━━━ 20s 125ms/step


<h2>Classification Report</h2>

In [32]:
print(classification_report(validation_generator.classes, y_pred, target_names=['defective', 'non-defective']))

               precision    recall  f1-score   support

    defective       0.20      0.20      0.20      1000
non-defective       0.80      0.80      0.80      4000

     accuracy                           0.68      5000
    macro avg       0.50      0.50      0.50      5000
 weighted avg       0.68      0.68      0.68      5000



<h2>Saving Model</h2>

In [47]:
model.save('Bridge_crack_detection_model.h5')

In [48]:
from tensorflow.keras.models import load_model

In [49]:
load_model = load_model('Bridge_crack_detection_model.h5')
load_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,034,179 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)